## Copies data from blob to local host

##### Copyright (C) Microsoft Corporation.  
see license file for details 

In [11]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# AZUREML_NATIVE_SHARE_DIRECTORY mapping to host dir is set by _nativeSharedDirectory_ in .compute file 

import os
try:
    amlWBSharedDir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']    
except:
    amlWBSharedDir = ''
    print('not using aml services?')
    
amlWBSharedDir

'/azureml-share/'

In [13]:
# Use the Azure Machine Learning data collector to log various metrics

from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [14]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [15]:
# import utlity functions

import sys, os
paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['Code',  'src'])))]
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

[None]

In [16]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

os.getcwd()


data_base_input_dir=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
credential_info_path=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.CREDENTIALS_DIR_list)))
nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

!mkdir -p {data_base_input_dir}
!mkdir -p {credential_info_path}  
!mkdir -p {nih_chest_xray_data_dir}

data_base_input_dir
credential_info_path
nih_chest_xray_data_dir

'/azureml-run'

'/azureml-share/chestxray/data/ChestX-ray8'

'/azureml-share/code/notShared'

'/azureml-share/chestxray/data/ChestX-ray8/ChestXray-NIHCC'

In [17]:
# Create this function in file '{credential_info_path}/get_data_access_secrets.py'
# def get_blob_credentials():
#     dataBlob = 'https://somedatastore.blob.core.windows.net/somecontainer'
#     sourceKey = 'somesourceKey'
#     return dataBlob, sourceKey
# 
# For example:
CredentialsFileName = os.path.join(credential_info_path, 'get_data_access_secrets.py') 
crt_container = 'https://somedatastore.blob.core.windows.net/somecontainer' 
crt_source_Key = 'somesourceKey'
import os.path 
if not os.path.isfile(CredentialsFileName): 
    print('Credentials file not found, will be written!') 
    with open(CredentialsFileName,'w') as myFile:
        myFile.write(("def get_blob_credentials():\n\t" +
        "dataBlob = '" + crt_container + "'\n\t" +
        "sourceKey = '" + crt_source_Key + "'\n\t" +
        "return dataBlob, sourceKey\n" )) 
    print(('Credentials file written at ' +CredentialsFileName)) 
else: 
    print(('Credentials file found at ' + CredentialsFileName))
    
# check CredentialsFileName existence and print content if needed

credentials_file_name = os.path.join(*([credential_info_path, 'get_data_access_secrets.py']))
credentials_file_name
# !cat {credentials_file_name}

Credentials file found at /azureml-share/code/notShared/get_data_access_secrets.py


'/azureml-share/code/notShared/get_data_access_secrets.py'

In [18]:
# read credentials info

import sys
def prepend_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
prepend_path_to_sys_path(credential_info_path)         
import get_data_access_secrets   
crt_container, crt_key = get_data_access_secrets.get_blob_credentials()

In [19]:
#check azcopy is available. Uncomment second line to print _response_ if needed

response = !azcopy
# response

In [20]:
# copy data to local host dir
# add {  --source-key {crt_key}  } if needed

answer = !yes | azcopy \
    --source {crt_container} \
    --destination {nih_chest_xray_data_dir} \
    --recursive

In [21]:
answer[-7:]

['[2018/02/06 05:19:05] Transfer summary:',
 '-----------------',
 'Total files transferred: 112120',
 'Transfer successfully:   112120',
 'Transfer skipped:        0',
 'Transfer failed:         0',
 'Elapsed time:            00.00:09:57']

In [22]:
nih_chest_xray_data_dir
!find $nih_chest_xray_data_dir -type f | wc -l

'/azureml-share/chestxray/data/ChestX-ray8/ChestXray-NIHCC'

112120


In [23]:
#  run below command in amlwb cli to save current sessin as html
# jupyter nbconvert --to html .\Code\01_DataPrep\001_get_data.ipynb